In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

# 유방암 데이터셋 로드
data = datasets.load_breast_cancer()
df = pd.DataFrame(data.data, columns = data.feature_names)

# 일부 feature만 사용
df = df[['mean radius', 'mean texture', 'mean area', 'mean symmetry']]

# 1이면 양성 종양, 0이면 악성 종양 라벨 열 지정
df['target'] = data.target

df

,mean radius,mean texture,mean area,mean symmetry,target
0,17.99,10.38,1001.0,0.2419,0
1,20.57,17.77,1326.0,0.1812,0
2,19.69,21.25,1203.0,0.2069,0
3,11.42,20.38,386.1,0.2597,0
4,20.29,14.34,1297.0,0.1809,0
...,...,...,...,...,...
564,21.56,22.39,1479.0,0.1726,0
565,20.13,28.25,1261.0,0.1752,0
566,16.60,28.08,858.1,0.1590,0
567,20.60,29.33,1265.0,0.2397,0


In [13]:
from sklearn.model_selection import train_test_split

# 독립변수와 종속변수 지정, 
x = df[['mean radius', 'mean texture', 'mean area', 'mean symmetry']] # 독립변수
y = df['target'] # 종속변수

# train, test 셋 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [23]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 학습
model = LogisticRegression(penalty = 'l2')
model.fit(x_train, y_train) # 머신러닝 목적에 맞게 훈련시킨다.

print('train dataset accuracy : %.2f' % model.score(x_train, y_train))
print('test dataset accuracy : %.2f' % model.score(x_test, y_test))

train dataset accuracy : 0.89
test dataset accuracy : 0.89


In [42]:
pred_y = model.predict_proba(x_test)[:, 1]
pred_y

array([8.05431938e-05, 3.69535281e-06, 9.93002889e-01, 2.10080190e-02,
       9.53241174e-01, 8.20767289e-04, 8.22866547e-01, 9.56985856e-01,
       6.74579420e-01, 1.84637682e-02, 5.17757290e-06, 2.10507376e-05,
       9.78172766e-01, 4.47553075e-02, 9.98022562e-01, 9.93561087e-01,
       9.99442865e-01, 9.87105717e-01, 9.67225127e-01, 9.95488916e-01,
       3.81511144e-02, 9.97314302e-01, 6.42246756e-01, 2.49986432e-01,
       9.98692884e-01, 8.86867705e-01, 7.78007429e-01, 8.23558090e-01,
       9.86838815e-01, 9.95797151e-01, 9.49386066e-01, 5.60182348e-01,
       9.99590752e-01, 6.13814733e-08, 8.12877865e-01, 9.93285559e-01,
       2.22962797e-01, 9.85279664e-03, 2.60801834e-04, 6.53051704e-01,
       1.37077287e-01, 9.54580547e-01, 9.98691287e-01, 3.47935794e-01,
       2.93975916e-01, 4.73503460e-01, 9.69602378e-01, 4.31842887e-04,
       9.99916940e-01, 9.08240921e-01, 7.63414435e-02, 9.79573066e-03,
       3.73001446e-04, 1.35822116e-02, 5.46549656e-04, 8.03474999e-05,
      

#### LogisticRegression 파라미터
참고 : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

- penalty : 패널티 정도를 지정한다. 독립 변수를 없애 간명하게 만들어 과적합 현상을 없애려는 것 (default: l2)
    - 'l1', 'l2', 'elasticnet', 'none'
- C : penalty에 대한 계수 설정, 높을 수록 복잡한 모델에 대한 규제 강화
- solver : 최적화 문제에 사용할 알고리즘, 선택한 패널티에 따라 다르게 적용한다.
    - 'newton-cg' [‘l2’, ‘none’]
    - 'lbfgs' [‘l2’, ‘none’]
    - 'liblinear' [‘l1’, ‘l2’]
    - 'sag' ['l2', 'none']
    - 'saga' [‘elasticnet’, ‘l1’, ‘l2’, ‘none’]
    - 'sag'와 'saga'는 스케일이 거의 동일한 기능에서만 보장된다.

In [15]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86        42
           1       0.92      0.92      0.92        72

    accuracy                           0.89       114
   macro avg       0.89      0.89      0.89       114
weighted avg       0.89      0.89      0.89       114



조금 더 자세한 평가를 위해 classification_report 모듈을 이용한다.

이 모듈은 모형 성능평가를 위한 모듈
정밀도(precision), 재현율(recall), F1-score, support를 구해준다.

In [16]:
# 로지스틱 회귀분석 학습
trains_cols = df.columns[1:] # train_cols는 독립(설명)변수
logit = sm.Logit(df[['target']],x)

# 로지스틱 회귀모델을 적합 시킬 때에 사용하는 최적화 기법 지정
result = logit.fit(method = 'newton')

Optimization terminated successfully.
         Current function value: 0.201859
         Iterations 9


In [17]:
## 회귀계수 확인
result.params

mean radius       3.354949
mean texture     -0.238953
mean area        -0.053430
mean symmetry   -48.417914
dtype: float64

In [18]:
##summary 함수로 결과 확인
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.694      
Dependent Variable: target           AIC:              237.7160   
Date:               2022-05-24 18:53 BIC:              255.0915   
No. Observations:   569              Log-Likelihood:   -114.86    
Df Model:           3                LL-Null:          -375.72    
Df Residuals:       565              LLR p-value:      9.3445e-113
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     9.0000                                        
------------------------------------------------------------------
                 Coef.   Std.Err.    z    P>|z|   [0.025   0.975] 
------------------------------------------------------------------
mean radius       3.3549   0.3513  9.5509 0.0000   2.6665   4.0434
mean texture     -0.2390   0.0425 -5.6183 0.0000  -0.3223  -0.1556
mean area        -0.0534   0.0054 -9.9148 0.0000  -0.0640  -0.0429
mean symmetry   -48.4179   6.9653 -6.9513 0.0000 -62.0696 -34.7662
==================================================================

"""

각 회귀계수가 합격 여부에 어떻게 영향을 미치는지 확인한다. 먼저, 유의확률(P>|z|) 즉 P-value를 본다. 
P-value의 기본값인 0.05 이하인 경우 유의미한 값으로 보기 때문에 학점 변수가 종속변수에 영향을 주
는 유의미한 변수임을 알 수 있다. 입시점수를 제외하고 계속해서 해석을 진행한다. 편회귀계수(Coef.)의 
부호를 통해 종속변수에 미치는 영향의 방향을 파악할 수 있다. 편회귀계수(Coef.)의 값이 양수라면 합격
여부가 ‘1’일 확률이 높아진다는 뜻이다. 반대로 음수라면 합격 여부의 값이 ‘0’일 확률이 높아진다는 뜻이
다. 따라서 모형의 회귀계수 분석결과는 다음과 같다. 통계량에 나와있는 편회귀계수의 값 자체만으로는 
각 변수들이 종속변수에 얼마나 영향을 주는지는 파악할 수 없다.

유의확률 : 유의확률(p-value, p값)은 귀무가설이 진실일 때 적어도 그 정도의 극단적인 표본값이 나올 확률, 즉 귀무가설이 참임에도 이를 기각할 확률  
유의수준 : 유의수준은 귀무가설의 기각여부를 결정하는데 사용하는 기준이 되는 확률  
귀무가설 : 설정한 가설이 진실일 확률이 극히 적어 처음부터 버릴 것이 예상되는 가설  
편회귀계수 : 회귀식에 포함되는 다른 변수의 영향을 제거한 후 독립변수가 종속변수에 주는 영향을 나타낸다. 원인(독립변수)이 여러 개인 다중회귀분석에서의 회귀계수를 의미

In [19]:
#종속변수에 미치는 정도는 오즈비(Odds Ratio, 승산비)를 통해 파악할 수 있다.
np.exp(result.params)

mean radius      2.864415e+01
mean texture     7.874519e-01
mean area        9.479722e-01
mean symmetry    9.383545e-22
dtype: float64

In [40]:
##cut_off 함수를 생성하여 임계치(threshold) 설정
##로지스틱은 확률값을 표현하기 때문에 임계치를 정해줘야 함
##일반적으로 임계치를 0.5로 정하여, 0.5 이상이면 1로, 아니면 0으로 판단
def cut_off(y, threshold):
    Y = y.copy() #대문자 Y를 새로운 변수로 지정하여 기존의 y값에 영향을 주지 않도록 함
    Y[Y > threshold] = 1
    Y[Y <= threshold] = 0
    return(Y.astype(int))

pred_Y = cut_off(pred_y, 0.5)

##confusion_matrix 함수로 혼동행렬을 뽑는다
cfmat = confusion_matrix(y_test, pred_Y)
print(cfmat)

[[38  4]
 [ 6 66]]
